In [1]:
# Import needed libraries 

import pandas as pd 
import os
import datetime
from datetime import timedelta
import numpy as np 
from scipy.signal import argrelextrema
import alpaca_trade_api as tradeapi 
import matplotlib.pyplot as plt 
import matplotlib.dates as mpdates
from mplfinance.original_flavor import candlestick_ohlc
from dotenv import load_dotenv
from itertools import islice

In [2]:
# Load .env file

load_dotenv()

True

In [3]:
# Set Alpaca API key and secret passwords

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [4]:
# Initiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [5]:
# Parameters for Stock Data from Alpacas
# Establish time frame (5 minute)

time_frame = "5min"

# Identify what stock symbol is trading

stock_symbol = "QQQ"

# Identify what start date to begin data analysis
# JAN 27 27 2021 GameStop event (GME) are we still doing this?

start_date = pd.Timestamp("2021-02-05", tz="America/New_York").isoformat()

# Identify what end date to finalize data analysis

end_date = pd.Timestamp("2021-02-13", tz="America/New_York").isoformat()

In [6]:
# Function to call daily stock data

def get_stock_data(api, stock_symbol, time_frame, current_date_iso, next_day_date_iso):
    
    # Assuming api.get_bars returns a DataFrame with a 'df' attribute
    
    stock_data = api.get_bars(
        stock_symbol, 
        time_frame, 
        start=current_date_iso, 
        end=next_day_date_iso
        ).df
    
    return stock_data

stock_data = get_stock_data(api, stock_symbol, time_frame, start_date, end_date)

# Displays the information pulled for working through code

stock_data.info()
display(stock_data.head())
display(stock_data.tail())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1092 entries, 2021-02-05 09:00:00+00:00 to 2021-02-12 22:55:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        1092 non-null   float64
 1   high         1092 non-null   float64
 2   low          1092 non-null   float64
 3   trade_count  1092 non-null   int64  
 4   open         1092 non-null   float64
 5   volume       1092 non-null   int64  
 6   vwap         1092 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 68.2 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-02-05 09:00:00+00:00,330.91,331.16,330.79,23,331.16,2057,330.941799
2021-02-05 09:05:00+00:00,330.90,330.95,330.90,74,330.92,7007,330.914585
2021-02-05 09:10:00+00:00,330.94,331.00,330.93,14,330.93,1753,330.954136
2021-02-05 09:15:00+00:00,330.80,330.98,330.76,46,330.96,10888,330.808653
2021-02-05 09:20:00+00:00,330.84,330.86,330.77,14,330.77,1577,330.795865


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-02-12 22:35:00+00:00,336.82,336.82,336.82,7,336.82,3293,336.483167
2021-02-12 22:40:00+00:00,336.78,336.80,336.78,5,336.80,1009,336.789316
2021-02-12 22:45:00+00:00,336.80,336.80,336.78,5,336.78,1060,336.789302
2021-02-12 22:50:00+00:00,336.80,336.80,336.80,16,336.80,680,336.793339
2021-02-12 22:55:00+00:00,336.78,336.78,336.78,41,336.78,596,336.743859


In [7]:
# # Function to call daily stock data

# start_date = pd.Timestamp("2021-02-03", tz="America/New_York").isoformat()

# # Identify what end date to finalize data analysis

# end_date = pd.Timestamp("2021-02-04", tz="America/New_York").isoformat()

# def get_stock_data(api, stock_symbol, time_frame, current_date_iso, next_day_date_iso):
    
#     # Assuming api.get_bars returns a DataFrame with a 'df' attribute
    
#     stock_data = api.get_bars(
#         stock_symbol, 
#         time_frame, 
#         start=current_date_iso, 
#         end=next_day_date_iso
#         ).df
    
#     return stock_data

# stock_data_day_1 = get_stock_data(api, stock_symbol, time_frame, start_date, end_date)

# # Displays the information pulled for working through code

# stock_data_day_1.info()
# display(stock_data_day_1.head())
# # display(stock_data.tail())


In [8]:
# Function to prepare daily stock data to identify double top/bottom patterns and prepare for targets
# Includes establishing a polynomial fit and assigning new columns for localized min/max

# Polynomial Degree

polynomial_degree = 75

def polynomial_min_max_fit(stock_data, polynomial_degree):
    
    min_length = len(stock_data.index)
    x_data = np.arange(min_length)

    # Polynomial fitting
    polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
    polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
    polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
    polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)

    # Evaluate the polynomial for plotting
    y_polynomial_open = np.polyval(polynomial_coefficients_open, x_data)
    y_polynomial_high = np.polyval(polynomial_coefficients_high, x_data)
    y_polynomial_low = np.polyval(polynomial_coefficients_low, x_data)
    y_polynomial_close = np.polyval(polynomial_coefficients_close, x_data)

    # Identify local extrema for polynomial fit data (minima and maxima)
    local_poly_minima = argrelextrema(y_polynomial_close, np.less, order=5)[0]
    local_poly_maxima = argrelextrema(y_polynomial_close, np.greater, order=5)[0]

    # Convert the close price polynomial fit data into a dataframe
    # This is done for OHLC poly fit data

    poly_df = pd.DataFrame(y_polynomial_open)
    columns = ['poly_fit_open']
    poly_df.columns = columns
    poly_df = poly_df.assign(poly_fit_high = y_polynomial_high)
    poly_df = poly_df.assign(poly_fit_low = y_polynomial_low)
    poly_df = poly_df.assign(poly_fit_close = y_polynomial_close)

    # Reset the index of the original updated ticker dataframe to concat with the polynomial dataframe that does not include a timeseries
    # This will ensure that the indexed intergers of the ploynomial fit align with the time each data point corresponds to

    updated_stock_data = stock_data.reset_index()
    updated_stock_data = pd.concat([updated_stock_data, poly_df], axis='columns', join='inner')

    # Add minima and maxima column to the DataFrame

    updated_stock_data["minima"] = 0
    updated_stock_data["maxima"] = 0

    # Mark rows with local minima as 1 in the 'minima' column

    for index in local_poly_minima:
        updated_stock_data.at[index, 'minima'] = -1

    for index in local_poly_maxima:
        updated_stock_data.at[index, "maxima"] = 1

    # Create Target Columns - Double Top Target & Double Bottom Target
        
    updated_stock_data["dbl_top_target"] = 0
    updated_stock_data["dbl_bot_target"] = 0

    
    # del poly_df
    # del x_data
    # del local_poly_maxima
    # del local_poly_minima
    # del min_length
    # del columns
    
    #return updated_stock_data, min_length, y_polynomial_close, local_poly_minima, local_poly_maxima
    print("FINISHED")
    return updated_stock_data

#updated_stock_data, min_length, y_polynomial_close, local_poly_minima, local_poly_maxima = polynomial_min_max_fit(stock_data,polynomial_degree)

updated_stock_data = polynomial_min_max_fit(stock_data, polynomial_degree)


FINISHED


/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/numpy/lib/polynomial.py:667: RuntimeWarning: overflow encountered in multiply
  scale = NX.sqrt((lhs*lhs).sum(axis=0))
/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_7889/3767615901.py:14: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_7889/3767615901.py:15: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_7889/3767615901.py:16: RankWarning: Polyfit may be poorly conditione

In [9]:
updated_stock_data



,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
0,2021-02-05 09:00:00+00:00,330.91,331.16,330.79,23,331.16,2057,330.941799,330.874627,330.938282,330.732764,330.783934,0,0,0,0
1,2021-02-05 09:05:00+00:00,330.90,330.95,330.90,74,330.92,7007,330.914585,330.897766,330.963147,330.770500,330.827444,0,0,0,0
2,2021-02-05 09:10:00+00:00,330.94,331.00,330.93,14,330.93,1753,330.954136,330.923454,330.989904,330.809565,330.870928,0,0,0,0
3,2021-02-05 09:15:00+00:00,330.80,330.98,330.76,46,330.96,10888,330.808653,330.951107,331.018071,330.849425,330.914025,0,0,0,0
4,2021-02-05 09:20:00+00:00,330.84,330.86,330.77,14,330.77,1577,330.795865,330.980199,331.047211,330.889602,330.956414,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,2021-02-12 22:35:00+00:00,336.82,336.82,336.82,7,336.82,3293,336.483167,336.568662,336.584987,336.535594,336.575166,-1,0,0,0
1088,2021-02-12 22:40:00+00:00,336.78,336.80,336.78,5,336.80,1009,336.789316,336.576253,336.596020,336.554939,336.590755,0,0,0,0
1089,2021-02-12 22:45:00+00:00,336.80,336.80,336.78,5,336.78,1060,336.789302,336.643151,336.662580,336.637328,336.661606,0,0,0,0
1090,2021-02-12 22:50:00+00:00,336.80,336.80,336.80,16,336.80,680,336.793339,336.794648,336.806274,336.809129,336.809359,0,0,0,0


In [10]:
# # Plot data
# # Plot the stock data and identified minima

# plt.figure(figsize=(15, 5))
# plt.plot(stock_data.index, stock_data["close"], label='Close Prices', alpha=0.7)

# # # Plot polynomial fit

# plt.plot(stock_data.index[:min_length], y_polynomial_close, '-', markersize=1.0, color='black', alpha=0.9, label='Polynomial Fit')

# # # Plot red dots at local minima and blue dots at local maxima

# plt.scatter(stock_data.index[local_poly_minima], y_polynomial_close[local_poly_minima], color='red', label='Local Minima')
# plt.scatter(stock_data.index[local_poly_maxima],y_polynomial_close[local_poly_maxima], color="blue", label = "Local Maxima")

In [11]:
# Define the time independent DF for double top/bottom identification
# Timestamp remains a column, but can identify pattern from peak to peak immediately

def time_independent_data(updated_stock_data):

    

    time_independent_df = []
    time_independent_df = pd.DataFrame(time_independent_df, columns = ["timestamp","close","high","low","trade_count","open","volume","vwap","poly_fit_open","poly_fit_high","poly_fit_low","poly_fit_close","minima","maxima","dbl_top_target","dbl_bot_target"])

    for index, row in islice(updated_stock_data.iterrows(), 0, None):

        # Assign to rows only those that contain local min or max

        if (updated_stock_data.at[index,"minima"] == -1):
            time_independent_df.loc[index] = row
            a = updated_stock_data.iloc[index]["poly_fit_close"]
        elif (updated_stock_data.at[index,"maxima"] == 1):
            time_independent_df.loc[index] = row
            a = updated_stock_data.iloc[index]["poly_fit_close"]

    # Reset time dependent index
    # Set new time independent index

    time_independent_df.reset_index(inplace = True)
    time_independent_df.rename(columns={"index":"time_dependent_index"}, inplace = True)

    return time_independent_df

time_independent_df = time_independent_data(updated_stock_data)
time_independent_df.head(10)

,time_dependent_index,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
0,25,2021-02-05 11:05:00+00:00,331.5,331.6,331.42,90,331.42,23105,331.503387,331.409022,331.463242,331.369839,331.417299,0,1,0,0
1,77,2021-02-05 15:25:00+00:00,330.195,330.33,329.82,1879,329.89,192058,330.145647,330.775429,330.957627,330.587065,330.773897,-1,0,0,0
2,261,2021-02-08 16:55:00+00:00,332.24,332.2781,332.08,1181,332.1575,160314,332.179842,332.765255,332.908491,332.605546,332.764949,0,1,0,0
3,277,2021-02-08 18:15:00+00:00,332.473,332.51,332.15,1128,332.22,110552,332.35011,332.759303,332.89787,332.609853,332.762848,-1,0,0,0
4,414,2021-02-09 14:00:00+00:00,332.81,332.9,332.78,200,332.89,19729,332.824856,333.514629,333.619938,333.413077,333.515377,0,1,0,0
5,453,2021-02-09 17:15:00+00:00,333.855,334.09,333.85,1291,334.02,143807,333.966563,333.465211,333.613649,333.310859,333.466061,-1,0,0,0
6,562,2021-02-10 10:20:00+00:00,334.95,334.97,334.94,7,334.94,536,334.959758,334.658324,334.700615,334.607931,334.656416,0,1,0,0
7,674,2021-02-10 19:40:00+00:00,333.49,333.67,333.4,1897,333.4,231856,333.554606,332.232722,332.446257,332.023842,332.231096,-1,0,0,0
8,784,2021-02-11 13:05:00+00:00,334.25,334.25,334.17,78,334.17,11054,334.21183,334.215009,334.334055,334.091933,334.20771,0,1,0,0
9,853,2021-02-11 18:50:00+00:00,334.062,334.25,334.061,606,334.1199,63496,334.160637,333.901202,334.083074,333.669465,333.896531,-1,0,0,0


In [12]:
# Function to identify daily double top/bottom patterns
# Inherently there should only be 1 identification of the pattern as it is considered a trend reversal pattern
# The loop will break once the pattern is identified

def identify_double_patterns(time_independent_df, updated_stock_data):

    # Initialize variables to identify double top/bottom patterns
    # Time dependent variable x_0 will always begin at a local min/max which also coincides
    # With the start of a trend into a potential double top/bottom
    
    x_0 = 0

    # Initiation of double top/bottom variable
    
    a = 0

    # First peak/valley of double top/bottom pattern
    
    b = 0

    # Trough/peak of double top/bottom pattern

    c = 0

    # Second peak/vallye of double top/bottom pattern
    
    d = 0

    # Trigger of double top/bottom pattern
    
    e = 0

    # Final time increment to finalize and trigger double top signal
    
    x_f = 0

    # Read through code to identify double top/bottom and assign to target columns.

    for index, row in islice(time_independent_df.iterrows(), 0, len(time_independent_df) - 4):

        # Check for double top
        # If found then assigns x_f final time for writing to targets in time dependent dataframe

        if (time_independent_df.at[index,"minima"] == -1):
            a = time_independent_df.iloc[index]["poly_fit_close"]
            b = time_independent_df.iloc[index + 1]["poly_fit_close"]
            c = time_independent_df.iloc[index + 2]["poly_fit_close"]
            d = time_independent_df.iloc[index + 3]["poly_fit_close"]
            e = time_independent_df.iloc[index + 4]["poly_fit_close"]
            x_0 = time_independent_df.iloc[index]["time_dependent_index"]

            if (time_independent_df.iloc[index + 1]["poly_fit_low"]) < d < (time_independent_df.iloc[index + 1]["poly_fit_high"]) and (e < c):
                x_f = time_independent_df.iloc[index + 4]["time_dependent_index"]
                while (x_0 < x_f + 1):
                    updated_stock_data.at[x_0, "dbl_top_target"] = 1
                    x_0 = x_0 + 1
                break

        # Check for double bottom
        # If found then assigns x_f final time for writing to targets in time dependent dataframe
            
        elif (time_independent_df.at[index,"maxima"] == 1):
            a = time_independent_df.iloc[index]["poly_fit_close"]
            b = time_independent_df.iloc[index + 1]["poly_fit_close"]
            c = time_independent_df.iloc[index + 2]["poly_fit_close"]
            d = time_independent_df.iloc[index + 3]["poly_fit_close"]
            e = time_independent_df.iloc[index + 4]["poly_fit_close"]
            x_0 = time_independent_df.iloc[index]["time_dependent_index"] 

            if (time_independent_df.iloc[index+1]["poly_fit_low"]) < d < (time_independent_df.iloc[index+1]["poly_fit_high"]) and (e > c):
                x_f_min = time_independent_df.iloc[index + 4]["time_dependent_index"]
                while (x_0 < x_f + 1):
                    updated_stock_data.at[x_0, "dbl_bot_target"] = 1
                    x_0 = x_0 + 1
                    
                break
            
    return updated_stock_data, x_f

updated_stock_data, x_f = identify_double_patterns(time_independent_df,updated_stock_data)




In [13]:
# # Data Checking

# display(updated_stock_data.head(20))
# print(updated_stock_data.loc[x_f])

In [14]:
# Loop to collect and assess daily data one day at a time

current_date = datetime.datetime.fromisoformat(start_date)
next_day_date = current_date + datetime.timedelta(days=1)

master_df = {
    'timestamp': [],  # List of timestamps
    'close': [],      # List of close prices
    'high': [],       # List of high prices
    'low': [],        # List of low prices
    'trade_count': [], # List of trade counts
    'open': [],       # List of open prices
    'volume': [],     # List of volumes
    'vwap': [],       # List of volume-weighted average prices
    'poly_fit_open': [],    # List of polynomial fit open prices
    'poly_fit_high': [],    # List of polynomial fit high prices
    'poly_fit_low': [],     # List of polynomial fit low prices
    'poly_fit_close': [],   # List of polynomial fit close prices
    'minima': [],           # List of minima
    'maxima': [],           # List of maxima
    'dbl_top_target': [],   # List of double top targets
    'dbl_bot_target': []    # List of double bottom targets
}

daily_dataframes = []

master_df = pd.DataFrame(master_df)

print(current_date)
print(next_day_date)

while current_date <= pd.Timestamp(end_date):

    # Convert current_date to ISO format for API call

    current_date_iso = current_date.isoformat()

    # Calculate the next day

    next_day_date = current_date + datetime.timedelta(days=1)

    # Convert next_day_date to ISO format for API call

    next_day_date_iso = next_day_date.isoformat()

    # Fetch stock data for the current day

    stock_data = get_stock_data(api, stock_symbol, time_frame, current_date_iso, next_day_date_iso)
    
    if len(stock_data) == 0:
        next_day_date = current_date + datetime.timedelta(days=1)
        current_date = current_date + datetime.timedelta(days=1)
    
    stock_data = get_stock_data(api, stock_symbol, time_frame, current_date_iso, next_day_date_iso)
    # display(stock_data.head())
    updated_stock_data = polynomial_min_max_fit(stock_data, polynomial_degree)
    
    # print('FINISHED_2')

#, min_length, y_polynomial_close, local_poly_minima, local_poly_maxima
    time_independent_df = time_independent_data(updated_stock_data)

    updated_stock_data, x_f = identify_double_patterns(time_independent_df, updated_stock_data)

    # Perform your analysis or call your functions here
    # e.g., identify_double_patterns(time_independent_df, updated_stock_data)

    daily_dataframes.append(updated_stock_data)

    del stock_data
    del updated_stock_data
    del time_independent_df

    # Increment to the next day
    
    current_date = next_day_date

    master_df = pd.concat(daily_dataframes, ignore_index=True)


2021-02-05 00:00:00-05:00
2021-02-06 00:00:00-05:00
FINISHED


/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/numpy/lib/polynomial.py:667: RuntimeWarning: overflow encountered in multiply
  scale = NX.sqrt((lhs*lhs).sum(axis=0))
/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_7889/3767615901.py:14: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_7889/3767615901.py:15: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_7889/3767615901.py:16: RankWarning: Polyfit may be poorly conditione

KeyError: 'open'

In [ ]:
master_df.info()
print(len(master_df))
display(master_df.head(50))
display(master_df.tail(50))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 757 entries, 0 to 756
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   timestamp       757 non-null    datetime64[ns, UTC]
 1   close           757 non-null    float64            
 2   high            757 non-null    float64            
 3   low             757 non-null    float64            
 4   trade_count     757 non-null    int64              
 5   open            757 non-null    float64            
 6   volume          757 non-null    int64              
 7   vwap            757 non-null    float64            
 8   poly_fit_open   757 non-null    float64            
 9   poly_fit_high   757 non-null    float64            
 10  poly_fit_low    757 non-null    float64            
 11  poly_fit_close  757 non-null    float64            
 12  minima          757 non-null    int64              
 13  maxima          757 non-null    int

,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
0,2021-02-08 09:00:00+00:00,332.52,332.60,332.50,43,332.53,7077,332.551844,332.489847,332.608321,332.437811,332.525402,0,0,0,0
1,2021-02-08 09:05:00+00:00,332.58,332.62,332.47,26,332.53,5163,332.501669,332.639382,332.655588,332.628992,332.617645,0,1,0,0
2,2021-02-08 09:10:00+00:00,332.63,332.67,332.63,7,332.64,2992,332.638663,332.617743,332.629280,332.596660,332.595357,0,0,0,0
3,2021-02-08 09:15:00+00:00,332.60,332.65,332.56,15,332.60,2065,332.591991,332.547124,332.590469,332.500565,332.547376,0,0,0,0
4,2021-02-08 09:20:00+00:00,332.62,332.62,332.57,8,332.57,783,332.600204,332.488301,332.566537,332.420592,332.515015,0,0,0,0
5,2021-02-08 09:25:00+00:00,332.35,332.50,332.33,20,332.50,2942,332.450693,332.463852,332.564786,332.386712,332.510909,-1,0,0,0
6,2021-02-08 09:30:00+00:00,332.50,332.50,332.22,35,332.28,3173,332.361550,332.474574,332.581847,332.400230,332.532125,0,0,0,0
7,2021-02-08 09:35:00+00:00,332.58,332.59,332.52,14,332.53,3389,332.526117,332.510645,332.609905,332.448311,332.568886,0,0,0,0
8,2021-02-08 09:40:00+00:00,332.61,332.61,332.54,11,332.59,1287,332.580349,332.558838,332.640541,332.513419,332.610032,0,0,0,0
9,2021-02-08 09:45:00+00:00,332.59,332.67,332.59,14,332.65,3745,332.630774,332.606807,332.666861,332.578982,332.646104,0,0,0,0


,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
707,2021-02-11 20:35:00+00:00,333.7600,334.0550,333.7100,1187,334.0109,336420,333.161134,334.182106,334.434529,334.058591,334.239647,0,0,0,0
708,2021-02-11 20:40:00+00:00,333.7500,334.1000,333.5900,2107,333.7500,296751,333.843046,334.224324,334.481995,334.075942,334.263805,0,0,0,0
709,2021-02-11 20:45:00+00:00,334.1762,334.4500,333.7250,2652,333.7500,353254,334.179762,334.252480,334.509872,334.079482,334.274630,0,0,0,0
710,2021-02-11 20:50:00+00:00,334.3800,334.4300,333.9250,3058,334.1600,441740,334.138866,334.270249,334.521083,334.075831,334.277072,0,0,0,0
711,2021-02-11 20:55:00+00:00,334.5600,334.7900,334.2700,5252,334.4200,936499,334.095919,334.281958,334.519666,334.071966,334.276405,0,0,0,0
712,2021-02-11 21:00:00+00:00,334.4900,334.6600,334.4400,322,334.5700,492903,334.562730,334.291869,334.510149,334.074176,334.277406,0,0,0,0
713,2021-02-11 21:05:00+00:00,334.2700,334.5200,334.1700,281,334.5200,543243,334.498125,334.303462,334.496871,334.087096,334.283602,0,0,0,0
714,2021-02-11 21:10:00+00:00,334.4200,334.6188,334.2100,200,334.3600,86373,334.509941,334.318832,334.483349,334.112938,334.296697,0,0,0,0
715,2021-02-11 21:15:00+00:00,334.5800,334.6100,334.2700,123,334.3810,41755,334.491788,334.338310,334.471792,334.151068,334.316261,0,0,0,0
716,2021-02-11 21:20:00+00:00,334.5000,334.6000,334.4400,48,334.5800,9810,334.559693,334.360374,334.462844,334.198021,334.339787,0,0,0,0


In [ ]:
print(current_date)

2021-02-12 00:00:00-05:00
